# <center>Кодирование признаков</center>

In [13]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [14]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [15]:
# Выделим целевой столбец
X = data.drop(columns='price')
y = data['price']

Если применить линейную регрессию на сырых данных, то получим ошибку, что не удалось превратить строковое значение в число с плавающей точкой (*float*). Чтобы этой ошибки не возникало, необходимо закодировать данные.

Пример:
```python
lr = LinearRegression()
lr.fit(X,y)
```

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/8be15badcefe262082f2ba923930636a/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst3-ml6-3_2.png)

|Значение признака "Образование"|Ordinal Encoding|Binary Encoding|OneHot Encoding|
|-------------------------------|----------------|---------------|---------------|
|Нет|1|000|000001|
|Начальное|2|001|000010|
|Среднее|3|010|000100|
|BSc|4|011|001000|
|MSc|5|100|010000|
|PhD|6|101|100000|

Представленная ниже таблица показывает соответствие типа кодирования классу в `sklearn.processing`.

|Тип кодировки|Класс библиотеки sklearn|
|-------------|------------------------|
|Порядковое кодирование|`LabelEncoder`|
|Двоичное кодирование|`LabelBinarizer`|
|Однократное кодирование|`OneHotEncoder`|

Алгоритм кодирования в `sklearn` следующий:

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/b456ce89c7ba9b7599266ff9b61497ca/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst3-ml6-3_4.png)

Из предыдущих модулей мы знаем, что при решении задач машинного обучения данные разбираются на обучающую (*train*) и валидационную (*validation*) выборки (последняя также может быть тестовой (*test*) выборкой). По аналогии подгонка кодировщика происходит на обучающей выборке, а трансформация — на обучающей и на тестовой.

Почему так? Потому что наша обученная модель не должна видеть данные, которые подаются в неё на тесте. Только так мы можем судить о том, что модель обучена качественно. То же самое и с кодировкой.

Посмотрим на кодирование признака *Образование* способом «один-против-всех» (*one vs all*):

In [16]:
from sklearn.preprocessing  import LabelBinarizer
 
lb = LabelBinarizer()
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
lb.fit(education)
print('категории:', lb.classes_)
lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

> У класса `LabelBinarizer`, как и у двух остальных, есть атрибут `classes_`, который выводит список уникальных значений признака.

Важно производить кодирование номинальных признаков, даже если они уже представлены в числовом формате, так как, в отличие от порядковых признаков, **категории номинальных являются независимыми**. В случае порядкового кодирования таких признаков мы вносим искусственные закономерности в данные (например, чем больше числовой код цилиндров, тем лучше, хотя это необязательно так).

In [17]:
# Посмотрим на число уникальных значений номинальных признаков
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
    print('Число уникальных значений признака {}: '.format(column), data[column].nunique())

Число уникальных значений признака cylinders:  6
Число уникальных значений признака title_status:  5
Число уникальных значений признака transmission:  3
Число уникальных значений признака drive:  3
Число уникальных значений признака size:  4


Итак, нам подходит *однократное кодирование*. Применим его к выбранным столбцам. Так как у нас нет отдельной тестовой выборки, то мы используем только один метод — `fit_transform()`. В качестве аргумента передаём таблицу с выбранными для преобразования признаками.

С помощью метода `get_feature_names_out()` получим список новых названий колонок:

In [18]:
from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray()
 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


Отлично, категориальные признаки закодированы. Теперь нам необходимо соединить преобразованные данные с исходными.

>Напомним, что у библиотеки *pandas* есть дефолтный метод `get_dummies()` для получения однократного кодирования признаков. Однако `OneHotEncoder` способен принимать на вход как таблицы, так и *numpy*-массивы.

In [20]:
# Задание3.7

# Преобразуем полученный массив закодированных данных в формат DataFrame, явно указав имена колонок
data_onehot = pd.DataFrame(data_onehot, index=data.index, columns=column_names)

# Соединяем новую таблицу с исходной
data_new = pd.concat([data, data_onehot], axis=1)

# Удаляем закодированные столбцы columns_to_change из полученной таблицы
data_new = data_new.drop(columns=columns_to_change)

# Выведим на экран форму полученной таблицы
data_new.shape

(7017, 30)

# <center>Обработка пропусков и выбросов</center>

## <center>Работа с пропусками</center>